# The Start
Let's walk through the process of creating a vector index.

# Step 1: Load the Documents
The `IngestService` has a method `load_obsidian_notes` that loads the notes from an Obsidian vault.  The input can be either a list of Document objects or a directory containing Obsidian notes.

Using Obsidian notes provides several advantages:
- Notes are easy to write and edit.
- Frontmatter/tags transfer into the metadata of the nodes.
- The Headers provide natural splitting points for the text.

The `IngestService` class relies on Langchain's `ObsidianLoader` class to load the notes.  The `ObsidianLoader` class uses the frontmatter, tags, dataview fields, and file metadata to populate the metadata of the nodes.  The additional metadata will benefit retrieval of the best nodes to answer a given query.

The `load_obsidian_notes` method takes in either:
- a list of strings where each string is considered a markdown file.
- a directory path to an Obsidian vault.
The list of strings options is useful for testing.

In [8]:
# E.g. to show using the ability to read in a list of Documents to test out loading Documents.

doc = """#Calcium_additive #raise_ph #Wollastonite #Silicon_additive #buffer_pH #Calcium
Growers  turn to Wollastonite for:
- Its **liming** capability.  Wollastonite's dissolution rate is slower than agricultural lime, offering a buffering effect against rapid pH changes. This makes Wollastonite beneficial in areas with fluctuating acidity levels.
- Adding **Silicon**.
- Adding **Calcium**.
Wollastonite's pH buffering effect and Silicon content contribute to pest control and powdery mildew suppression, although the exact mechanisms are not fully understood.

# What is Wollastonite?

## Formation
Wollastonite is formed when Limestone is subjected to heat and pressure during metamorphism if surrounding silicate minerals are present.
### Basic Reaction:
Given high pressure and high temperature:
- CaCO3 (Limestone) + SiO2 (silica) → CaSiO3 (Wollastonite) + CO2 (carbon Dioxide)
## Sources
China is the largest producer of Wollastonite. Other areas where Wollastonite is mined include the United States (although it was originally mined in California, the only active mining in the U.S. is now in New York State), India, Mexico, Canada, and Finland.

## Industrial Applications of Wollastonite

|Industry|Application|
|---|---|
|Ceramics|Smoother and more durable ceramics, reinforcement agent|
|Plastics and Rubber|Cost-effective strengthening agent|
|Paints and Coatings|Reinforcement, improved durability and impact resistance|
|Construction|Improved strength and durability of building materials, safe alternative to asbestos|
##  How Wollastonite Provides Plants with Ca and Si

Wollastonite reacts with Water and Carbon Dioxide in the soil to form Calcium Bicarbonate and Silicon Dioxide.
- CaSiO₃ (Wollastonite)+2CO₂ (carbon Dioxide,)+H₂O (Water)→Ca(HCO₃)₂ (Calcium bicarbonate)+SiO₂ (silica)

### Calcium
- Calcium bicarbonate  (Ca(HCO₃)₂) is unstable and fairly easily decomposes to Limestone (CaCO₃):
		- Ca(HCO₃)₂ (Calcium bicarbonate)→CaCO₃ (Limestone)+  CO₂ (carbon Dioxide) + H₂O (Water)

- Soils with a pH below 7 (acidic soils) contain hydrogen ions (H+). These hydrogen ions react with the Limestone (CaCO3) to form Calcium ions (Ca2+), Water (H2O), and Carbon Dioxide (CO2).
	- CaCO3 (Limestone) + 2H+ (hydrogen ions) → Ca2+ (Calcium ions) + H2O (Water) + CO2 (carbon Dioxide)
### Silicon
- Silicon Dioxide slowly breaks down into Silicic Acid, which plants absorb. This process is influenced by soil pH, temperature, and microbial activity.
	- SiO2 (Silicon Dioxide) + 2H2O (Water) → H4SiO4 (Silicic Acid)

- Plants absorb Silicic Acid from the soil solution through their roots.


"""

In [ ]:
# This notebook is in the eval folder.  Change to the root folder.
%cd ..
%pwd  # To verify the current working directory

In [ ]:
# --->: Read in the markdown files in the Obsidian vault directory
from src.ingest_service import IngestService
from src.doc_stats import DocStats
# The Directory containing the knowledge documents used by the AI to do the analysis on the soil tests.
soil_knowledge_directory = r"G:\My Drive\Audios_To_Knowledge\knowledge\AskGrowBuddy\AskGrowBuddy\Knowledge\soil_test_knowlege"
# Load the documents
ingest_service = IngestService()
loaded_documents = ingest_service.load_obsidian_notes(soil_knowledge_directory)
# Show some summary stats about the documents

DocStats.print_llama_index_docs_summary_stats(loaded_documents)

In [ ]:
from node_view import launch_node_viewer
# Create and launch the interface
launch_node_viewer(loaded_documents)

# Step 2: Split the Documents using Markdown Splitting

In [ ]:
text_nodes = ingest_service.chunk_text(loaded_documents)
DocStats.print_llama_index_docs_summary_stats(text_nodes)

# Step 3: Delete Unuseful Nodes
Some nodes will not contain any useful content. I delete them to provide cleaner data to the retriver.  I also check to see what other challenges might be occuring. In one case, I noted Excalidraw drawings were included.  I filtered these out.

In [ ]:
from node_view import launch_node_viewer
# Create and launch the interface
launch_node_viewer(text_nodes)


In [8]:
# Saving the nodes in case we want to start before indexing.
import pickle
with open('eval/text_nodes.pkl', 'wb') as f:
    pickle.dump(text_nodes, f)

In [6]:
# Now unpickle
import pickle

with open('eval/text_nodes.pkl', 'rb') as f:
    text_nodes = pickle.load(f)

In [1]:
from rich import print


In [ ]:
import chromadb
client = chromadb.PersistentClient(path="vectorstore")
collections = client.list_collections()
print(collections)

# Step 4: Build the Index
Now onto building the vector index.  We will use methods in `IngestService`.  We will start by setting up Ollama.  Ollama is used both for embedding as well as retrieval/inference.

`chromadb` is used as the persistent store for the vector index.

In [ ]:
from src.ingest_service import IngestService
ingest_service = IngestService()
# Create a Chroma collection object of a given name. Metadata, embeddings, text are all added.
our_collection = ingest_service.build_vector_index(nodes=text_nodes, collection_name='soil_test_knowledge', embed_model_name='multi-qa-mpnet-base-cos-v1')


Now we can always get to our persisted index.

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
client = chromadb.PersistentClient(path="vectorstore")
collection_name = "soil_test_knowledge"
embedding_function = SentenceTransformerEmbeddingFunction(model_name='multi-qa-mpnet-base-cos-v1')
our_collection = client.get_collection(collection_name, embedding_function=embedding_function)


Let's retrieve some nodes and see how they look.

In [ ]:
query = 'what is the ideal pH for cannabis?'
results_dict = our_collection.query(
    query_texts=[query],
    n_results=3,

)
print(f"Text of first result: {results_dict["documents"][0][0]}")
print(f"Similarity of first result: {results_dict["distances"][0][0]}")

I do not understand the results. I expected them to be close to 1 since the query sentences is very similar to the text of the node.

In [ ]:
from src.visualize import Visualize
visualize = Visualize()
iface = visualize.plot_3d_plotly()
iface.launch()